In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import pandas as pd

### First distance formula test (unused)

In [ ]:
# First attempt at a distance formula. Found this python module that does exactly what we want, taking into account curvature of the earth, and its oblateness


# import geopy.distance

# coords_1 = (40.603995, -73.755405)
# coords_2 = (40.600066, -73.761353)

# print(geopy.distance.distance(coords_1, coords_2).miles)

### Set up webdriver and set parameters for page scraping

In [ ]:
# URL for mta alerts in table format
url = "https://www.mymtaalerts.com/messagearchive.aspx"

In [ ]:
# create a new Edge session
driver = webdriver.Edge()
driver.implicitly_wait(30)
driver.get(url)

# grab date input elements and set them to the time frame that we want to look at
date_from = driver.find_element('id', 'ctl00_ContentPlaceHolder1_dtpStartDate_dateInput')
date_from.clear()
date_from.send_keys('01/01/2018')
date_to = driver.find_element('id','ctl00_ContentPlaceHolder1_dtpStopDate_dateInput')
date_to.clear()
date_to.send_keys('01/01/2022')

# grab agency type input (subway,buses, etc) and select only subway alerts
agency = driver.find_element('id','ctl00_ContentPlaceHolder1_ddlAgency')
agency.send_keys('NYCT Subway')

# click elevator/escalator checkbox so that we do not scrape alerts involving elevators or escalators
elevator = driver.find_element('id','ctl00_ContentPlaceHolder1_chkHideElevatorEscalator')
elevator.click()
update = driver.find_element('id','ctl00_ContentPlaceHolder1_btnGetData')
update.click()

### Start scraping with BeautifulSoup

In [ ]:
last_page = 1000 
"""
This is the number of pages that you want to scrape.
last_page can be found on the website after selecting date range, agency type, and elevator/escalator checkboxes
"""

alerts_time = [] # append parsed alerts here

for i in range(last_page):
    # grab html and put it into bs4
    src = driver.page_source
    html = BeautifulSoup(src,'html.parser')

    # grab all rows from the table for this specific page
    table = html.find_all('tbody')[-1]
    rows = table.find_all('tr')

    for row in rows:
        cols = row.find_all('td')
        alert = {}
        alert['id'] = cols[0].string
        datetime = cols[1].contents
        alert['datetime'] = str(datetime[0])+" "+str(datetime[2])
        alert['agency'] = cols[2].string
        alert['title'] = cols[3].string
        alert['message'] = cols[4].string
        alerts_time.append(alert)


    current_page = html.find('a', attrs={'class':'rgCurrentPage'}).text

    print(current_page) # prints out current page number to check progress


    # next_page button changes name after 10 pages, and then changes back on the last page. This switches its name.
    if int(current_page) < 11 or int(current_page) == last_page:
        next_page = driver.find_element('name','ctl00$ContentPlaceHolder1$gridMessages$ctl00$ctl02$ctl01$ctl18')
    else:
        next_page = driver.find_element('name','ctl00$ContentPlaceHolder1$gridMessages$ctl00$ctl02$ctl01$ctl20')

    next_page.click()

### Change to a dataframe and save as csv

In [ ]:
alerts_pd = pd.DataFrame.from_dict(alerts_time)

In [ ]:
alerts_pd.to_csv('alerts_2022.csv')

### Upload this csv to blob storage